In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, beta, gamma
from scipy.stats import multivariate_normal as mn
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [17]:
rawdata = pd.read_csv('gapminder.tsv.txt', sep='\t')
life_exp = rawdata['lifeExp']
continent = pd.get_dummies(rawdata['continent'],drop_first=True,prefix="continent")
continent+=1
data = rawdata[['year', 'country', 'pop', 'gdpPercap']]
data = pd.concat([data, continent], axis=1)

data

,year,country,pop,gdpPercap,continent_Americas,continent_Asia,continent_Europe,continent_Oceania
0,1952,Afghanistan,8425333,779.445314,1,2,1,1
1,1957,Afghanistan,9240934,820.853030,1,2,1,1
2,1962,Afghanistan,10267083,853.100710,1,2,1,1
3,1967,Afghanistan,11537966,836.197138,1,2,1,1
4,1972,Afghanistan,13079460,739.981106,1,2,1,1
...,...,...,...,...,...,...,...,...
1699,1987,Zimbabwe,9216418,706.157306,1,1,1,1
1700,1992,Zimbabwe,10704340,693.420786,1,1,1,1
1701,1997,Zimbabwe,11404948,792.449960,1,1,1,1
1702,2002,Zimbabwe,11926563,672.038623,1,1,1,1


In [3]:
def get_X(data):
    X = np.zeros((len(data), len(data.columns)), dtype=float)
    for i, row in data.iterrows():
        for j in range(len(row)):
            X[i][j] = row[j]
            
    XTX = np.linalg.inv(np.matmul(X.T, X))
    XTy = np.matmul(X.T, life_exp)
    beta_ols = np.matmul(XTX, XTy)
    
    return beta_ols

In [18]:
countries = rawdata['country'].unique()
betas = []

for country in countries:
    print(country)
    dat = data[data['country']==country]
    dat=dat.drop(['country'], axis=1)
    dat = dat.reset_index()
    b = get_X(dat)
    betas.append(b)

Afghanistan


LinAlgError: Singular matrix

In [81]:
d = data[data['country']=='Brazil']
y = np.array(rawdata[rawdata['country']=='Brazil']['lifeExp'])
d = d.drop(['country'], axis=1)
d = d.reset_index()
d = d.drop(['index'], axis=1)

X = np.zeros((len(d), len(d.columns)), dtype=float)
for i, row in d.iterrows():
    for j in range(len(row)):
        X[i][j] = row[j]

(7, 12)

# create gibbs sampler

v = np.matmul((life_exp-np.matmul(X, beta_ols)), (life_exp-np.matmul(X, beta_ols)))/(len(data)-8)
sigma = v*np.matmul(X.T, X)
sigma0=sigma*np.eye(7)

var = np.linalg.inv(np.linalg.inv(sigma0)+np.matmul(np.matmul(X.T, X), np.linalg.inv(sigma)))

E_second = np.matmul(np.linalg.inv(sigma0), beta_ols)+np.matmul(np.matmul(X.T, life_exp), np.linalg.inv(sigma))
E = np.matmul(var, E_second)

# begin gibbs sampler
N=10000
burn=1000
beta_post = []
beta=beta_ols

for i in range(N):
    # update beta and get ssr(beta)
    v = np.matmul((life_exp-np.matmul(X, beta)), (life_exp-np.matmul(X, beta)))/(len(data)-8)
    sigma = v*np.linalg.inv(np.matmul(X.T, X))
    if np.all(np.linalg.eigvals(var) >= 0):
        beta = mn(E, var, allow_singular=True).rvs()
    
    var = np.linalg.inv(np.linalg.inv(sigma0)+np.matmul(np.matmul(X.T, X), np.linalg.inv(sigma)))
    E_second = np.matmul(np.linalg.inv(sigma0), beta_ols)+np.matmul(np.matmul(X.T, life_exp), np.linalg.inv(sigma))
    E = np.matmul(var, E_second)
    
    if i>=burn:
        beta_post.append(beta)